In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

# 超参数

BATCH_SIZE = 32
LR = 0.01
# greedy policy
EPSILON = 0.9
# reward discount
GAMMA = 0.9
# 目标网络更新频率
TARGET_REPLACE_ITER = 100
# 记忆库容量
MEMORY_CAPACITY = 2000
# CartPole-v0环境，render_mode="human"意味着环境会以一种适合人类直接观察的方式进行渲染，比如弹出一个窗口来实时展示环境中的场景
env = gym.make('CartPole-v1', render_mode="human").unwrapped
# 杆子动作的个数，本环境中是离散动作，即只有向左向右
N_ACTIONS = env.action_space.n
# Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
# [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]中分别代表：小车位置的最小值、小车速度的最小值、杆子角度的最小值、杆子角速度的最小值
# [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]代表最大值
# env.observation_space.shape[0]代表元组的长度，此处用N_STATES代表状态数4，仅仅恰好相等
N_STATES = env.observation_space.shape[0]

In [ ]:
class Net(nn.Module):
    def __init__(self):
        # nn.Module的子类函数必须在构造函数中执行父类的构造函数
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(N_STATES, 50)
        self.fc1.weight.data.normal_(0, 0.1)
        self.fc2 = nn.Linear(50, N_ACTIONS)
        self.fc2.weight.data.normal_(0, 0.1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)
    
class DQN(object):
    def __init__(self):
        # 创建DQN的两个网络，评估和目标网络
        self.eval_net, self.target_net = Net(), Net()
        # 
        self.learn_step_counter = 0
        #
        self.memory_counter = 0
        # 经验回放记忆库，N_STATES * 2 + 2是指 s, a, r, s'
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))
        # 优化器
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()
        
    def choose_action(self, x):
        # (4,)变为(1, 4)
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        if np.random.uniform() < EPSILON :
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0]
        else:
            action = np.random.randint(0, N_ACTIONS)
        return action
    
    def store_transition(self, s, a, r, s_):
        # np.hstack(([1, 2, 3, 4], [0, 1], [5, 6, 7, 8]) = [1, 2, 3, 4, 0, 1, 5, 6, 7, 8]
        transition = np.hstack((s, [a, r], s_))
        # 记忆库满了
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1
        
    def learn(self):
        # 每TARGET_REPLACE_ITER轮更新参数
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1
        
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])
        
        q_eval = self.eval_net(b_s).gather(b_a)
        # detach将该张量从计算图分离，它不会再参与梯度的计算
        q_next = self.target_net(b_s_).detach()
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)
        loss = self.loss_func(q_eval, q_next)
        

TypeError: 'Discrete' object is not subscriptable